In [77]:
# imports
import os

import openai  # for calling the OpenAI API
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from dotenv import load_dotenv
from IPython.Display import Markdown, display

# Doc ingestion


# openai
openai.api_type = "azure"
openai.api_base =  os.getenv('OPENAI_API_ENDPOINT')
openai.api_version = "2023-03-15-preview"

openai.api_key = os.getenv('OPENAI_API_KEY')
formrecognizer_key=os.getenv('FORM_RECOGNIZER_KEY')
formrecognizer_endpoint=os.getenv('FORM_RECOGNIZER_ENDPOINT')

openai.api_key = os.getenv('OPENAI_API_KEY')

load_dotenv() 
searchservice=os.getenv("AZURE_SEARCH_SERVICE")
search_creds = AzureKeyCredential(os.getenv("AZURE_SEARCH_KEY"))
search_index=os.getenv("AZURE_SEARCH_INDEX")

In [78]:
default_messages=[{"role":"system","content":"You are a librarian assistant."}]

def setup_engine(engine,messages):
    engine = openai.ChatCompletion.create(
    engine=engine,
    messages = messages,
    temperature=0.7,
    max_tokens=800,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None)
    return engine

def return_response(message, engine,message_log=False):
    if not message_log:
        new_messages=default_messages.copy()
    else:
        new_messages=message_log.copy()
    new_messages.append({"role":"user","content":message})
    response=setup_engine(engine,new_messages)
    return response, new_messages

In [79]:
def generate_search_query_text(question):
    query_prompt_template = """You are a search term librarian helping a user that needs to be answered by searching in a knowledge base about Industrial Psychology.
            Generate a search query based on the conversation and the new question.
            If the question is not in English, translate the question to English before generating the search query.

        Question:
        {question}

        Search query:
        """
    prompt = query_prompt_template.format(
                question=question
            )
    # convert content to string
    response,message_history=return_response(prompt,"gpt35")
    return response.choices[0]["message"]["content"]

def get_search_results(search_query, search_client,verbose=False):
    r=search_client.search(search_query, top=8)
    results = [
                "REFERENCE: " + doc['reference'] + "\nSECTION TEXT:\n " +
                doc['sectiontext'] + " \n END OF SECTION.\n"
                for doc in r
            ]
    content = "\n".join(results)
    if verbose:
        print(content)
    return content

search_client = SearchClient(
        endpoint=f"https://{searchservice}.search.windows.net/",
        index_name=search_index,
        credential=search_creds,
    )

In [83]:
max_prompt = """You are Max, Max helps industrial organizational doctoral students prepare for their comprehensive exams and generates outlines to answer domain questions.
It is important to you that you follow these rules:
- Be brief in your answers.
- ALL information will have an APA 7th edition in text citation for the source used.
- All outlines will have a references section at the end.
- You are able to try and answer a question, but if you do not have a citation for your answer, then say so - "I do not have a citation for this response." 
- Do NOT make up citations.
- Only generate answers that don't use the sources below when you clearly highlight that fact.
- If asking a clarifying question to the user would help, ask the question.
- Responses will use a moderate level GRE vocabulary
- Responses will use a graduate level writing style, appropriate for a doctoral student in the domain of industrial psychology.
- IF a question does not have a scenario THEN keep the answer simple and answer the main constructs; do not give outside examples or scenarios
- IF a question does have a scenario THEN answer to the issues or concerns relevant to the scenario AND use constructs from research to answer
- Give all formatting in Markdown, not HTML.

IMPORTANT: All responses must be compliant with the criteria:

- Did the response answer all parts of the original domain question?
- Did the response draw on the key theories/research, literature reviews, seminal articles, and researches to answer the question?
- Did the response build cogent and integrated response to the question?

- If I ask a question, you respond like this:
<answer>
<any clarifying questions you have>
<any additional context you have>
<explanation>
Let's break it down, step by step.

Sources:
{sources}
"""

In [99]:
def ask(query,verbose=False):
    search_query=generate_search_query_text(query)
    if verbose:
        print("Search query: ",search_query)
    results=get_search_results(search_query,search_client,verbose)
    if verbose:
        print("Search results: ",results)
    prompt=max_prompt.format(sources=results)
    message_log=[{"role":"system","content":prompt},{"role":"user","content":query}]
    if verbose:
        print(message_log)
    response,message_history=return_response(prompt,"gpt4", message_log)
    return response.choices[0]["message"]["content"]

def markdown_ask(query,verbose=False):
    response=ask(query,verbose)
    display(Markdown(response))

In [96]:
markdown_ask("What is boundaryless and protean careers?")

Boundaryless career refers to a career that transcends traditional organizational boundaries, such as departmental identity and job duties that are ambiguous (Miner & Robinson, 1994), and encompasses physical and/or psychological mobility (Sullivan & Arthur, 2006). Protean career, on the other hand, emphasizes a self-directed approach to the career that is driven by personal values (Briscoe & Hall, 2006) and involves competencies in "knowing-why," "knowing-how," and "knowing-whom" (Guan, Arthur, & Khapova, 2019). Combining the two concepts results in 16 possible career profiles that vary in values-driven and self-directedness, as well as psychological and physical mobility (Briscoe & Hall, 2006).

In [100]:
markdown_ask("Write an outline for an essay on boundaryless and protean careers")